In [140]:
# Import necessary modules
# glob allows unix style pathname pattern expansion
from glob import glob
# pydicom is the python dicom reader
import pydicom as dicom
import nibabel.nicom.csareader as csareader
import os
import pandas as pd

In [141]:
# 5-T1w_MPR_vNav -- MPRAGE structrual dicoms
# 6-fMRI_DistortionMap_PA -- fmri field map dicoms
# 7-fMRI_DistortionMap_AP -- fmri field map dicoms opposite phase encode direction
# 9-fMRI_REVL_ROI_loc_2 -- fmri localizer dicoms run1
# 10-fMRI_REVL_Study_1 -- fmri task dicoms run1
# 14-dMRI_DistortionMap_AP_dMRI_REVL -- dwi field map dicoms
# 16-dMRI_AP_REVL -- diffusion weighted dicoms

# directories will have to be specific to your computer
base_dir = f'/home/sjsuss626/Mattfeld_PSB6351'
proj_dir = '/sourcedata'
subj_dir = '/Mattfeld_REVL-000-vCAT-021-S1/scans/10-fMRI_REVL_Study_1/resources/DICOM/files'

# with strings I can concatenate them with simple addition lines
data_dir = base_dir + proj_dir + subj_dir

# Here I use glob to grab the dicom files
# Why would I use glob?  What does it give me?
data_files = glob(data_dir + '/*')


In [142]:
data_files

len(data_files)

355

In [144]:
func_vol

Dataset.file_meta -------------------------------
(0002,0000) File Meta Information Group Length  UL: 204
(0002,0001) File Meta Information Version       OB: b'\x00\x01'
(0002,0002) Media Storage SOP Class UID         UI: MR Image Storage
(0002,0003) Media Storage SOP Instance UID      UI: 1.3.12.2.1107.5.2.43.166003.2019061915555358347291073
(0002,0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002,0012) Implementation Class UID            UI: 1.2.40.0.13.1.1
(0002,0013) Implementation Version Name         SH: 'dcm4che-2.0'
(0002,0016) Source Application Entity Title     AE: 'AN_MEDCOMNT204'
-------------------------------------------------
(0008,0005) Specific Character Set              CS: 'ISO_IR 100'
(0008,0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'M', 'ND', 'MOSAIC']
(0008,0012) Instance Creation Date              DA: '20190619'
(0008,0013) Instance Creation Time              TM: '155555.436000'
(0008,0016) SOP Class UID      

In [145]:
dir(func_vol)
#dir(func_vol['InPlanePhaseEncodingDirection'])
#func_vol['InPlanePhaseEncodingDirection'].value

['AccessionNumber',
 'AcquisitionDate',
 'AcquisitionMatrix',
 'AcquisitionNumber',
 'AcquisitionTime',
 'AngioFlag',
 'BitsAllocated',
 'BitsStored',
 'BodyPartExamined',
 'Columns',
 'CommentsOnThePerformedProcedureStep',
 'ContentDate',
 'ContentTime',
 'DeidentificationMethodCodeSequence',
 'DeviceSerialNumber',
 'EchoNumbers',
 'EchoTime',
 'EchoTrainLength',
 'FlipAngle',
 'FrameOfReferenceUID',
 'HighBit',
 'ImageOrientationPatient',
 'ImagePositionPatient',
 'ImageType',
 'ImagedNucleus',
 'ImagingFrequency',
 'InPlanePhaseEncodingDirection',
 'InstanceCreationDate',
 'InstanceCreationTime',
 'InstanceNumber',
 'InstitutionAddress',
 'InstitutionName',
 'InstitutionalDepartmentName',
 'LargestImagePixelValue',
 'MRAcquisitionType',
 'MagneticFieldStrength',
 'Manufacturer',
 'ManufacturerModelName',
 'Modality',
 'NumberOfAverages',
 'NumberOfPhaseEncodingSteps',
 'OperatorsName',
 'PatientAge',
 'PatientBirthDate',
 'PatientComments',
 'PatientID',
 'PatientName',
 'PatientPos

In [146]:
TR=func_vol['RepetitionTime'].value
TE=func_vol['EchoTime'].value
FA=func_vol['FlipAngle'].value
DIR=func_vol['InPlanePhaseEncodingDirection'].value
FOV=func_vol[0x0018, 0x0094].value

In [159]:
# to find the slice timing, we will calculate it from TR and number of slices

def calculate_slice_timing(func_vol):
    """Calculates slice timing for a series of DICOM files.

    Args:
        dicom_files (list): A list of DICOM file paths.

    Returns:
        slice_timing (float): The slice timing value.
    """

    # Extract relevant information from the first DICOM file
    ds = dicom.dcmread(data_files[0])

    # Get the number of slices by total number of dicom files in scan session
    num_slices = len(data_files)

    # Extract TR
    tr = float(ds.RepetitionTime)  # Repetition Time in milliseconds

    # first calculate aquition time for whole scan
    ta = tr - (tr / num_slices)

    # Calculate time per slice
    slice_timing = ta / num_slices

    #Multiply by 3 because of multiband (s3)
    slice_timing = slice_timing*3

    return slice_timing


slice_timing = calculate_slice_timing(func_vol)
print(slice_timing)

14.831342987502477


In [160]:
# Elements we're interested in
# Repetition Time
# Echo Time
# Acquisition Matrix 
# Flip Angle
# Acquisition Number
# Protocol Name
# Slice times = func_vol[0x00191029].value

print(f'The number of TRs collected equals: {len(data_files)}')
print(f'The timing of the TR was: {TR} ms')
print(f'The timing of the TE was: {TE} ms')
print(f'The flip angle in degrees was: {FA}')
print(f'The name of the scan was: Mattfeld_REVL-000-vCAT-021-S1')
print(f'The phase encode direction was: {DIR}')
print(f'COL = AP or PA; ROW = RL or LR')
print(f'The slice timing was: {slice_timing}')
print(f'The field of view - FOV was: {FOV}')

# Why use the following format to access information in the header?
#print(func_vol[int('0051100c', 16)].value) # func_vol[0x0051100c].value
# Allows us to access information that is private in the dicom header.

# How do I find phase encoding direction information?

for elem in func_vol:
    if "Phase" in elem.name or "Direction" in elem.name or "Encoding" in elem.name:
        print(f"{elem.tag}: {elem.name}: {elem.value}")

print('1 = +; 0 = -')
print('COL, 1 = PA; COL, 0 = AP')

The number of TRs collected equals: 355
The timing of the TR was: 1760 ms
The timing of the TE was: 35 ms
The flip angle in degrees was: 52
The name of the scan was: Mattfeld_REVL-000-vCAT-021-S1
The phase encode direction was: COL
COL = AP or PA; ROW = RL or LR
The slice timing was: 14.831342987502477
The field of view - FOV was: 100
(0018,0089): Number of Phase Encoding Steps: 90
(0018,0094): Percent Phase Field of View: 100
(0018,1312): In-plane Phase Encoding Direction: COL
(0019,1028): [BandwidthPerPixelPhaseEncode]: 16.393
(0051,1013): [PositivePCSDirections]: +LPH
1 = +; 0 = -
COL, 1 = PA; COL, 0 = AP
